In [1]:
import numpy as np
from signals import get_price, get_price_rsi, extrema_lines, slope_filter, concativity_filter, line_point_filter, timeframe_multiplier, timeframe_divider
from backtesting import trail_stop



In [2]:
from scipy.signal import find_peaks
import numpy as np
import plotly.graph_objects as go
from signals.rsi import rsi

import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
import plotly.graph_objects as go


In [3]:

class StrategyParser():
    def __init__(self, start, end, lines_settings, values_settings, trade_settings, verbose = False):
        '''
        timeframe(int): how long in minutes a candle period is
        '''
        self.lines_settings = lines_settings
        self.values_settings = values_settings
        self.trade_settings = trade_settings
        self.verbose = verbose
        
        self.values_by_name = {}
        self.lines_by_name = {}
        self.trade_by_name = {}
        
        self.start = start
        self.end = end
        
        
        self.org = "Artinus"
        url = "https://eastus-1.azure.cloud2.influxdata.com"
        token = "l7mZ63vVVzgUKP2z6_XMnZWKKJ4VNEeA6hSMPbqj_-z2l86OpSV8RA9EMzwoTPZSgg9KZzA8R6AkwztiBHgT6A=="

        self.client = influxdb_client.InfluxDBClient(url=url, token=token, org=self.org)
        self.bucket = 'DukascopyTest'
        
    def get_values(self,name, value_type,**settings):
        # {'value_type':'price','period':5, 'measurement': 'bid_low'}
        if value_type == 'price':
            self.values_by_name[name] = get_price(self.client, self.org, self.bucket, self.start, self.end,**settings)
        if value_type == 'rsi':
            self.values_by_name[name] = get_price_rsi(self.client, self.org, self.bucket, self.start, self.end,**settings)[1]
        
        
    def recursive_line(self, line_setting = None, name = None):
        '''
        
        line_setting: can be a dict that defines lines or a string of the lines name
        '''
        _id = ''
        
        if isinstance(line_setting, str):
            # print(line_setting)
            name = line_setting
            line_setting = self.lines_settings[line_setting]
        
        
        if name is not None and name in self.lines_by_name:
            if self.verbose:
                print(f'PRECOMPUTED:{name}')
            return self.lines_by_name[name], line_setting
        if self.verbose:
                print(f'START:{name}')
        
        if 'line_point_filter' in line_setting:
            if self.verbose:
                print('\tline_point_filter')
            
            
            left, line_setting_left = self.recursive_line(line_setting = line_setting['line_point_filter']['left'])
            right, line_setting_right = self.recursive_line(line_setting = line_setting['line_point_filter']['right'])
            
            # converts the left and right lines to have comparable peroid indecies
            lcm = np.lcm(line_setting_left['period'],line_setting_right['period'])
            
            left_multiplier = lcm//line_setting_right['period']
            right_multiplier = lcm//line_setting_left['period']

            left = timeframe_multiplier(left, left_multiplier)
            right = timeframe_multiplier(right, right_multiplier)
            
            if isinstance(line_setting['line_point_filter']['filter_type'], int):
                left, right = line_point_filter(left, 
                                                right, 
                                                filter_type = line_setting['line_point_filter']['filter_type'], 
                                                allowed_offset = line_setting['line_point_filter']['allowed_offset'])
                
            else:
                left, right = line_point_filter(left, 
                                                right, 
                                                filter_type = line_setting['line_point_filter']['filter_type'][1], 
                                                allowed_offset = line_setting['line_point_filter']['allowed_offset'][1])
                
                left, right = line_point_filter(left, 
                                                right, 
                                                filter_type = line_setting['line_point_filter']['filter_type'][0], 
                                                allowed_offset = line_setting['line_point_filter']['allowed_offset'][0])
                
                
            # converts the desired line back to original peroid indecies 
            # if line_setting['line_point_filter']['keep'] == 'last':
                
                
                
            if line_setting['line_point_filter']['keep'] == 'right':
                lines = timeframe_divider(right, right_multiplier)
                line_setting['period'] = line_setting_right['period']
                
            if line_setting['line_point_filter']['keep'] == 'left':
                lines = timeframe_divider(left, left_multiplier)
                line_setting['period'] = line_setting_left['period']
            else:
                pass
            
        
        if 'values' in line_setting:
            if self.verbose:
                print('\tvalues')
            values = self.values_by_name[line_setting['values']]
            line_setting['period'] = self.values_settings[line_setting['values']]['period']
            
        if 'lines' in line_setting:
            if self.verbose:
                print('\tlines')
            lines, old_line_setting= self.recursive_line(line_setting = line_setting['lines'])
            line_setting['period'] = old_line_setting['period']
            line_setting['_print_delay'] = old_line_setting['_print_delay']
            
        if 'extrema_lines' in line_setting:
            if self.verbose:
                print('\textrema_lines')
            lines = extrema_lines(values,**line_setting['extrema_lines'])
            line_setting['_print_delay'] = 2*line_setting['period']
                
        if 'slope_filter' in line_setting:
            if self.verbose:
                print('\tslope_filter')
            lines = slope_filter(values, lines, **line_setting['slope_filter'])
            
        if 'concativity_filter' in line_setting:
            if self.verbose:
                print('\tconcativity_filter')
            lines = concativity_filter(values, lines, **line_setting['concativity_filter'])
        
        if name is not None:
            self.lines_by_name[name] = lines
        if self.verbose:
            print(f'END:{name}')
        return lines, line_setting
    
    
    def trade(self, trade_setting, name):
        if self.verbose:
            print(f'START:{name}')
        self.trade_by_name[name] = {}
        trade = self.trade_by_name[name]
        if 'trail_stop' in trade_setting:
            
            if self.verbose:
                print('\ttrail_stop')
                
            values = self.values_by_name[trade_setting['trail_stop']['values']]
            lines = self.lines_by_name[trade_setting['trail_stop']['lines']]

            s = {key:trade_setting['trail_stop'][key] for key in trade_setting['trail_stop'] if (key!='lines' and key!='values')} 

            trade['entry_x'], trade['entry_y'], trade['exit_x'], trade['exit_y'], trade['ratio'] = trail_stop(values, lines, **s)
        if self.verbose:
            print(f'END:{name}')


    def run(self):
        for values_setting in self.values_settings:
            self.get_values(values_setting, **self.values_settings[values_setting])
            
        for name in self.lines_settings.keys():
            self.recursive_line(self.lines_settings[name], name)
            
        for name in self.trade_settings.keys():
            self.trade(self.trade_settings[name], name)
    def get_stats(self):
        stats = {}
        for name in self.trade_by_name:
            if len(self.trade_by_name[name]['ratio']) == 0:
                stats[name] = None
            else:
                stats[name] = {}
                stats[name]['avg_ratio'] = np.mean(self.trade_by_name[name]['ratio'])
                stats[name]['trade_count'] = len(self.trade_by_name[name]['ratio'])
                stats[name]['sum_log_ratio'] = np.sum(np.log(self.trade_by_name[name]['ratio']))
                stats[name]['avg_log_ratio'] = np.mean(np.log(self.trade_by_name[name]['ratio']))
            
        return stats
    
    def print_stats(self):
        stats = self.get_stats()
        for name in stats:
            print(stats[name])
        
    def plot(self, values_name, lines_name=None, trade_name= None):
        values = self.values_by_name[values_name]
        
        
        
        
        data=[
            go.Scattergl(
                y=values, mode="markers+lines", name=values_name
            ),
        ]
        
        if lines_name is not None:
            lines = self.lines_by_name[lines_name]
            x_plot_lines = []
            y_plot_lines = []
            for line in lines:
                x_plot_lines += [line[0],line[1], None]
                y_plot_lines += [values[line[0]], values[line[1]], None]
                
            data+= [
                go.Scattergl(
                    x=x_plot_lines, y=y_plot_lines, mode="markers", name=lines_name+' markers'
                ),
                go.Scattergl(
                    x=x_plot_lines, y=y_plot_lines, mode="markers+lines", name=lines_name
                ),
            ]
            
            
        if trade_name is not None:
            x = []
            y = []
            trade = self.trade_by_name[trade_name]
            entry_x, entry_y, exit_x, exit_y = trade['entry_x'], trade['entry_y'], trade['exit_x'], trade['exit_y']
            for i in range(0,len(entry_x)):
                x += [entry_x[i],exit_x[i],None]
                y += [entry_y[i],exit_y[i],None]
                
            data+= [
                go.Scattergl(
                    x=x_plot_lines, y=y_plot_lines, mode="markers+lines", name=lines_name
                ),
            ]

        fig = go.Figure(
                data=data
            )
        fig.show()

In [4]:
settings = {
    'start': '2022-11-09T01:00:00+00:00',
    'end': '2022-12-01T01:00:00+00:00',
    'values_settings':{
        '5m_low':{'value_type':'price','period':5, 'measurement': 'bid_low'},
        '5m_rsi':{'value_type':'rsi','period':5, 'measurement': 'bid_close'},
        '1m_low':{'value_type':'price','period':1, 'measurement': 'bid_low'},
        '1m_rsi':{'value_type':'rsi','period':1, 'measurement': 'bid_close'},
        
    },

    'lines_settings':{
        'Bull Reverse Divergence': {
            'values': '5m_low',
            'slope_filter': {'filter_type': 'higher'},
            'lines':{
                'values': '5m_rsi',
                'extrema_lines': {'filter_type': 'low', 'num_periods': 200, 'height': 30},
                'slope_filter': {'filter_type': 'lower'},
                'concativity_filter': {'filter_type': "convex"}
            }
        },

        'Bull Regular Divergence': {
                'values': '1m_low',
                'slope_filter': {'filter_type': 'lower'},
                'lines': {
                    'values': '1m_rsi',
                    'extrema_lines': {'filter_type': 'low', 'num_periods': 28, 'height': 30},
                    'slope_filter': {'filter_type': 'higher'},
                    'concativity_filter': {'filter_type': "convex"}
                }
        },
        'Reloaded': {
            'line_point_filter': {
                'filter_type': 1,
                'allowed_offset': 15, 
                'keep' : 'right',
                'left' : 'Bull Reverse Divergence',
                'right': 'Bull Regular Divergence'
            }
        }
    },
    'trade_settings':{
        'Reloaded':{
            "trail_stop":{
                'lines': 'Reloaded',
                'values': '1m_low',
                'trail_threshold':0.001,
                'line_index': 1, 
                'short' : False,
                'period_delay': 2
            }
        }
    }
}

p = StrategyParser(**settings, verbose = True)
p.run()

START:Bull Reverse Divergence
	values
	lines
START:None
	values
	extrema_lines
	slope_filter
	concativity_filter
END:None
	slope_filter
END:Bull Reverse Divergence
START:Bull Regular Divergence
	values
	lines
START:None
	values
	extrema_lines
	slope_filter
	concativity_filter
END:None
	slope_filter
END:Bull Regular Divergence
START:Reloaded
	line_point_filter
PRECOMPUTED:Bull Reverse Divergence
PRECOMPUTED:Bull Regular Divergence
END:Reloaded
START:Reloaded
	trail_stop
END:Reloaded


In [5]:
p.plot('1m_low', 'Reloaded', 'Reloaded')
p.print_stats()

{'avg_ratio': 0.9991611240917295, 'trade_count': 4, 'sum_log_ratio': -0.003356926426969999, 'avg_log_ratio': -0.0008392316067424997}


In [6]:


import time
from multiprocessing import Pool
import copy

import dateutil.relativedelta as relativedelta
import dateutil.rrule as rrule
import datetime

def run_strat(settings):
    p = StrategyParser(**settings)
    p.run()
    return p.get_stats()

s = []
before=datetime.datetime(2015,1,1)
after=datetime.datetime(2022,2,1)

rr = rrule.rrule(rrule.WEEKLY,byweekday=relativedelta.MO,dtstart=before)
for monday in rr.between(before,after,inc=True):
    monday+= datetime.timedelta(minutes=1)
    friday = monday+ datetime.timedelta(days=5)
    
    monday = monday.isoformat()+'+00:00'
    friday = friday.isoformat()+'+00:00'
    temp_settings = copy.deepcopy(settings)
    temp_settings['start'], temp_settings['end'] = monday, friday
    s.append(temp_settings)
# print(s)
with Pool(32) as p:
    stats = p.map(run_strat, s)

    


In [7]:
name = 'Reloaded'
total_trade_count = np.sum([stat[name]['trade_count']  for stat in stats if stat[name] is not None])

avg_ratio = 0
sum_log_ratio = 0
avg_log_ratio = 0

for stat in stats[1:]:
    if stat[name] is not None:
        trade_count = stat[name]['trade_count']
        avg_ratio += stat[name]['avg_ratio']*(trade_count/total_trade_count)
        sum_log_ratio += stat[name]['sum_log_ratio']
        avg_log_ratio += stat[name]['avg_log_ratio']*(trade_count/total_trade_count)
        
print(f'total_trade_count: {total_trade_count}, avg_ratio: {avg_ratio}, sum_log_ratio: {sum_log_ratio}, avg_log_ratio: {avg_log_ratio}')

total_trade_count: 425, avg_ratio: 0.9999069353436971, sum_log_ratio: -0.03980265691878765, avg_log_ratio: -9.365331039714736e-05


In [8]:
np.e**sum_log_ratio

0.9609790630105046